In [1121]:
import pandas as pd
import numpy as np
import networkx as nx
import plotly.graph_objects as go
import plotly.offline as pyo
import random


In [1122]:
# Load Dataframes from csv files
avg_counts_df = pd.read_csv('avg_counts_df.csv')
avg_rates_df = pd.read_csv('avg_rates_df.csv')
player_df = avg_counts_df.copy()
player_df

,Unnamed: 0,[0-0],[1-0],[0-1],[2-0],[1-1],[0-2],[3-0],[2-1],[1-2],[3-1],[2-2],[3-2],OUT,WALK,PLAY
0,[0-0],0.0,69861.0,91708.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,20010.0
1,[1-0],0.0,0.0,0.0,23620.0,35117.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,10919.0
2,[0-1],0.0,0.0,0.0,0.0,36510.0,39148.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,15716.0
3,[2-0],0.0,0.0,0.0,0.0,0.0,0.0,7236.0,12357.0,0.0,0.0,0.0,0.0,0.0,0.0,3907.0
4,[1-1],0.0,0.0,0.0,0.0,0.0,0.0,0.0,24406.0,32622.0,0.0,0.0,0.0,0.0,0.0,14320.0
5,[0-2],0.0,0.0,0.0,0.0,0.0,9238.0,0.0,0.0,21450.0,0.0,0.0,0.0,8652.0,0.0,8788.0
6,[3-0],0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,4473.0,0.0,0.0,0.0,2363.0,359.0
7,[2-1],0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,10756.0,17807.0,0.0,0.0,0.0,8089.0
8,[1-2],0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,15087.0,0.0,25876.0,0.0,13338.0,0.0,14493.0
9,[3-1],0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,7731.0,0.0,4160.0,3289.0


In [1123]:
# Round Average Rates to 2 decimal places

player_counts = [player_df.iloc[0]['[1-0]']+player_df.iloc[0]['[0-1]'],player_df.iloc[1]['[2-0]']+player_df.iloc[1]['[1-1]'],\
                 player_df.iloc[2]['[1-1]']+player_df.iloc[2]['[0-2]'],player_df.iloc[3]['[3-0]']+player_df.iloc[3]['[2-1]'],\
                 player_df.iloc[4]['[2-1]']+player_df.iloc[4]['[1-2]'],player_df.iloc[5]['[0-2]']+player_df.iloc[5]['[1-2]']+\
                 player_df.iloc[5]['OUT'],player_df.iloc[6]['[3-1]']+player_df.iloc[6]['WALK'],player_df.iloc[7]['[3-1]']+\
                 player_df.iloc[7]['[2-2]'],player_df.iloc[8]['[1-2]']+player_df.iloc[8]['[2-2]']+player_df.iloc[8]['OUT'],\
                 player_df.iloc[9]['[3-2]']+player_df.iloc[9]['WALK'],player_df.iloc[10]['[2-2]']+player_df.iloc[10]['[3-2]']+\
                 player_df.iloc[10]['OUT'],player_df.iloc[11]['[3-2]']+player_df.iloc[11]['OUT']+player_df.iloc[11]['WALK']]

avg_counts = [avg_counts_df.iloc[0]['[1-0]']+avg_counts_df.iloc[0]['[0-1]'],avg_counts_df.iloc[1]['[2-0]']+avg_counts_df.iloc[1]['[1-1]'],\
                 avg_counts_df.iloc[2]['[1-1]']+avg_counts_df.iloc[2]['[0-2]'],avg_counts_df.iloc[3]['[3-0]']+avg_counts_df.iloc[3]['[2-1]'],\
                 avg_counts_df.iloc[4]['[2-1]']+avg_counts_df.iloc[4]['[1-2]'],avg_counts_df.iloc[5]['[0-2]']+avg_counts_df.iloc[5]['[1-2]']+\
                 avg_counts_df.iloc[5]['OUT'],avg_counts_df.iloc[6]['[3-1]']+avg_counts_df.iloc[6]['WALK'],avg_counts_df.iloc[7]['[3-1]']+\
                 avg_counts_df.iloc[7]['[2-2]'],avg_counts_df.iloc[8]['[1-2]']+avg_counts_df.iloc[8]['[2-2]']+avg_counts_df.iloc[8]['OUT'],\
                 avg_counts_df.iloc[9]['[3-2]']+avg_counts_df.iloc[9]['WALK'],avg_counts_df.iloc[10]['[2-2]']+avg_counts_df.iloc[10]['[3-2]']+\
                 avg_counts_df.iloc[10]['OUT'],avg_counts_df.iloc[11]['[3-2]']+avg_counts_df.iloc[11]['OUT']+avg_counts_df.iloc[11]['WALK']]
player_counts = [int((i+random.uniform(-1000,1000))/100) for i in player_counts]
avg_counts = [int(i) for i in avg_counts]


In [1124]:
pct_avg_counts = [i/sum(avg_counts) for i in avg_counts]
pct_player_counts = [i/sum(player_counts) for i in player_counts]
count_diff = [(pct_player_counts[i]-pct_avg_counts[i])*100 for i in range(len(player_counts))]
count_diff

[-0.05267712577710637,
 0.08705916652074208,
 0.06091098979595644,
 0.06689143896646196,
 0.19097392596274243,
 0.02099723017288252,
 -0.12220141650095395,
 -0.007878939458229317,
 0.0038162402777791593,
 -0.07632370754480887,
 -0.10518213094789092,
 -0.06638567146757673]

In [1125]:
count_diff.append(0)
count_diff.append(0)
player_counts.append(0)
player_counts.append(0)

In [1126]:
player_counts = {
    '[0,0]': player_counts[0],
    '[1,0]': player_counts[1],
    '[0,1]': player_counts[2],
    '[2,0]': player_counts[3],
    '[1,1]': player_counts[4],
    '[0,2]': player_counts[5],
    '[3,0]': player_counts[6],
    '[2,1]': player_counts[7],
    '[1,2]': player_counts[8],
    '[3,1]': player_counts[9],
    '[2,2]': player_counts[10],
    '[3,2]': player_counts[11],
    'WALK': player_counts[12],
    'OUT': player_counts[13]
}
count_diff = {
    '[0,0]': count_diff[0],
    '[1,0]': count_diff[1],
    '[0,1]': count_diff[2],
    '[2,0]': count_diff[3],
    '[1,1]': count_diff[4],
    '[0,2]': count_diff[5],
    '[3,0]': count_diff[6],
    '[2,1]': count_diff[7],
    '[1,2]': count_diff[8],
    '[3,1]': count_diff[9],
    '[2,2]': count_diff[10],
    '[3,2]': count_diff[11],
    'WALK': count_diff[12],
    'OUT': count_diff[13]
}

In [1127]:
G = nx.DiGraph()
dec = 0
# Define edges between nodes that reflect state transition matrix
elist = [(1,2),(1,3),(2,4),(2,5),(3,5),(3,6),(4,7),(4,8),(5,8),(5,9),(6,6),(6,9),\
        (6,13),(7,10),(7,14),(8,10),(8,11),(9,9),(9,11),(9,13),(10,12),(10,14),\
        (11,11),(11,12),(11,13),(12,12),(12,13),(12,14)]



player_rates = [avg_rates_df.iloc[0]['[1-0]'],avg_rates_df.iloc[0]['[0-1]'],avg_rates_df.iloc[1]['[2-0]'],avg_rates_df.iloc[1]['[1-1]'],\
                 avg_rates_df.iloc[2]['[1-1]'],avg_rates_df.iloc[2]['[0-2]'],avg_rates_df.iloc[3]['[3-0]'],avg_rates_df.iloc[3]['[2-1]'],\
                 avg_rates_df.iloc[4]['[2-1]'],avg_rates_df.iloc[4]['[1-2]'],avg_rates_df.iloc[5]['[0-2]'],avg_rates_df.iloc[5]['[1-2]'],\
                 avg_rates_df.iloc[5]['OUT'],avg_rates_df.iloc[6]['[3-1]'],avg_rates_df.iloc[6]['WALK'],avg_rates_df.iloc[7]['[3-1]'],\
                 avg_rates_df.iloc[7]['[2-2]'],avg_rates_df.iloc[8]['[1-2]'],avg_rates_df.iloc[8]['[2-2]'],avg_rates_df.iloc[8]['OUT'],\
                 avg_rates_df.iloc[9]['[3-2]'],avg_rates_df.iloc[9]['WALK'],avg_rates_df.iloc[10]['[2-2]'],avg_rates_df.iloc[10]['[3-2]'],\
                 avg_rates_df.iloc[10]['OUT'],avg_rates_df.iloc[11]['[3-2]'],avg_rates_df.iloc[11]['OUT'],avg_rates_df.iloc[11]['WALK']]
player_rates = [element + random.uniform(-0.1, 0.1) for element in player_rates]

avg_rates = [avg_rates_df.iloc[0]['[1-0]'],avg_rates_df.iloc[0]['[0-1]'],avg_rates_df.iloc[1]['[2-0]'],avg_rates_df.iloc[1]['[1-1]'],\
              avg_rates_df.iloc[2]['[1-1]'],avg_rates_df.iloc[2]['[0-2]'],avg_rates_df.iloc[3]['[3-0]'],avg_rates_df.iloc[3]['[2-1]'],\
              avg_rates_df.iloc[4]['[2-1]'],avg_rates_df.iloc[4]['[1-2]'],avg_rates_df.iloc[5]['[0-2]'],avg_rates_df.iloc[5]['[1-2]'],\
              avg_rates_df.iloc[5]['OUT'],avg_rates_df.iloc[6]['[3-1]'],avg_rates_df.iloc[6]['WALK'],avg_rates_df.iloc[7]['[3-1]'],\
              avg_rates_df.iloc[7]['[2-2]'],avg_rates_df.iloc[8]['[1-2]'],avg_rates_df.iloc[8]['[2-2]'],avg_rates_df.iloc[8]['OUT'],\
              avg_rates_df.iloc[9]['[3-2]'],avg_rates_df.iloc[9]['WALK'],avg_rates_df.iloc[10]['[2-2]'],avg_rates_df.iloc[10]['[3-2]'],\
              avg_rates_df.iloc[10]['OUT'],avg_rates_df.iloc[11]['[3-2]'],avg_rates_df.iloc[11]['OUT'],avg_rates_df.iloc[11]['WALK']]

rate_diffs = [round((player_rates[i] - avg_rates[i])*100,dec) for i in range(len(player_rates))]


player_labels = [f"{int(round(element*100,dec))}%" for element in player_rates]
avg_labels = [f"{int(round(element*100,dec))}%" for element in avg_rates]

rate_labels = []

for rate_diff, player_rate in zip(rate_diffs, player_rates):
    if rate_diff > 0:
        rate_labels.append(f"+{int(round(rate_diff,dec))}%")
    else:
        rate_labels.append(f"{int(round(rate_diff,dec))}%")



edges_with_labels = [(elist[i][0], elist[i][1], {'label': rate_labels[i]}) for i in range(len(elist))]


In [1128]:

# Normalize rate_diffs to range between -1 and 1
normalized_diffs = 2 * (rate_diffs - np.min(rate_diffs)) / (np.max(rate_diffs) - np.min(rate_diffs)) - 1
normalized_diffs

color_intensities = []
for diff in normalized_diffs:
    color_intensities.append(int(255*diff))

In [1129]:
G.add_edges_from(edges_with_labels)
label_mapping = {1:'[0,0]',2:'[1,0]',3:'[0,1]',4:'[2,0]',5:'[1,1]',6:'[0,2]',\
                 7:'[3,0]',8:'[2,1]',9:'[1,2]',10:'[3,1]',11:'[2,2]',12:'[3,2]',\
                    13:'OUT',14:'WALK'}
G = nx.relabel_nodes(G, label_mapping)

In [1130]:
scale_factor = 1.5
pos = {
    '[0,0]': (150, 600*scale_factor),
    '[0,1]': (180, 500*scale_factor),
    '[1,0]': (120, 500*scale_factor),
    '[1,1]': (150, 400*scale_factor),
    '[0,2]': (210, 400*scale_factor),
    '[2,0]': (90, 400*scale_factor),
    '[1,2]': (180, 300*scale_factor),
    '[2,1]': (120, 300*scale_factor),
    '[2,2]': (150, 200*scale_factor),
    '[3,0]': (60, 300*scale_factor),
    '[3,1]': (90, 200*scale_factor),
    '[3,2]': (120, 100*scale_factor),
    'OUT': (210, 0),
    'WALK': (60, 0)
}

In [1131]:
# Create edge trace
def create_edge_trace(x0,y0,x1,y1,color,width):
    return  go.Scatter(
        x=[x0, x1, None],
        y=[y0, y1, None],
        mode='lines',
        line=dict(color=color, width=width),
        hoverinfo='none'
    )

edge_annotations = []

In [1132]:
edge_traces = []
loop_edge_colors = []
loop_text_colors = []
for i, edge in enumerate(G.edges()):
    x0, y0 = pos[edge[0]]
    x1, y1 = pos[edge[1]]
    if color_intensities[i] > 0:
        edge_color = f'rgba(255,{255-color_intensities[i]},{255-color_intensities[i]},1)'
        text_color = 'red'
    else:
        edge_color = f'rgba({255-abs(color_intensities[i])},{255-abs(color_intensities[i])},255,1)'
        text_color = 'blue'
    edge_trace = create_edge_trace(x0,y0,x1,y1,edge_color,width=3)
    edge_traces.append(edge_trace)

    if edge[0] == edge[1]:
        x_pos = x0+17
        y_pos = y0+5
        loop_edge_colors.append(edge_color)
        loop_text_colors.append(text_color)
    else:
        x_pos = (x0+x1)/2
        y_pos = (y0+y1)/2
    # Edge annotations (midpoint for label placement)
    edge_annotations.append(
        dict(
            x=x_pos,
            y=y_pos,
            xref='x',
            yref='y',
            text = '<b><i>'+str(G.edges[edge]['label'])+'</b></i>',
            showarrow=False,
            font=dict(color=text_color, size=14, family='Arial'),
            bgcolor='white',
            borderpad=0
        )
    )

In [1133]:
#static edge trace
edge_traces_single_color = []
for i, edge in enumerate(G.edges()):
    x0, y0 = pos[edge[0]]
    x1, y1 = pos[edge[1]]
    edge_color = 'rgba(0, 0, 0, 0.2)'  # Change the color here
    edge_trace_single_color = create_edge_trace(x0, y0, x1, y1, edge_color, width=3)
    edge_traces_single_color.append(edge_trace_single_color)


In [1134]:
# Create node trace
node_trace = go.Scatter(
    x=[],
    y=[],
    mode='markers+text',
    hoverinfo='none',
    marker=dict(
        showscale=False,
        color=[],
        size=[],
        symbol='circle',
        line_width=0.5,
        line_color='black',
        opacity=1),
    text=[],
    textposition="middle center",
    textfont=dict(
        family="Arial",
        size=14,
        color="black"
    )
)

node_diameter = 65
# Add nodes to node trace
for i,node in enumerate(G.nodes()):
    x,y = pos[node]
    node_trace['x'] += tuple([x])
    node_trace['y'] += tuple([y])
    # Customize node size and color
    node_trace['marker']['color'] += tuple(['rgb(220,220,220,1)'])
    node_trace['marker']['size'] += tuple([node_diameter + 100*count_diff[node]])
    if node == 'OUT' or node == 'WALK':
        node_trace['text'] += tuple(['<br></br>'+str(node)+'<br><i>'+str(player_counts[node])+'</i></br>'])
    else:
        node_trace['text'] += tuple(['<br></br>'+str(node)[1]+'-'+str(node)[-2]+'<br><i>'+str(player_counts[node])+'</i></br>'])


traces =  edge_traces + edge_traces_single_color + [node_trace]

In [1135]:
fig = go.Figure(data=traces,
                layout=go.Layout(
                    width=800,
                    height=850,
                    paper_bgcolor='rgba(0,0,0,0)',
                    plot_bgcolor='rgba(0,0,0,0)',
                    titlefont_size=16,
                    showlegend=False,
                    hovermode='closest',
                    margin=dict(b=20,l=5,r=5,t=40),
                    annotations=edge_annotations,
                    xaxis=dict(showgrid=False, zeroline=False, showticklabels=False, scaleanchor="y", scaleratio=5),
                    yaxis=dict(showgrid=False, zeroline=False, showticklabels=False, scaleanchor="x", scaleratio=1)))

In [1136]:
# Add loops to graph with arrow caps
loops = []
arrows = []

for i in range(4):
    loop = dict(
        start_pt = [pos[f'[{i},2]'][0]+8, pos[f'[{i},2]'][1]+20],
        end_pt = [pos[f'[{i},2]'][0]+10.5, pos[f'[{i},2]'][1]-7],
        control_pt1 = [pos[f'[{i},2]'][0]+40, pos[f'[{i},2]'][1]+23],
        control_pt2 = [pos[f'[{i},2]'][0]+15, pos[f'[{i},2]'][1]-25]
    )
    loops.append(loop)
    
    arrow = dict(
        type="path",
        path=f"M {loop['end_pt'][0]},{loop['end_pt'][1]-1} L {loop['end_pt'][0]+3},{loop['end_pt'][1]+1} L {loop['end_pt'][0]+2},{loop['end_pt'][1]-7} Z",
        fillcolor=loop_edge_colors[i],
        line=dict(color=loop_edge_colors[i])
    )
    arrows.append(arrow)
shapes = []
for i,loop in enumerate(loops):
    shapes.append(
        dict(
            type="path",
            path=f"M {loop['start_pt'][0]},{loop['start_pt'][1]} C {loop['control_pt1'][0]},{loop['control_pt1'][1]} {loop['control_pt2'][0]},{loop['control_pt2'][1]} {loop['end_pt'][0]},{loop['end_pt'][1]}",
            line_color=loop_edge_colors[i],
        )
    )

shapes.extend(arrows)

fig.update_layout(shapes=shapes)
# fig.show()

In [1137]:
from dash import Dash, html, dcc

In [1138]:

app = Dash(__name__)
app.layout = html.Div(children=[
    html.Div('Pitching with Markov', style={'font-family': 'Arial', 'font-size': '24px', 'font-style': 'italic'}),
    dcc.Graph(
        id='example-graph',
        figure=fig)
])


In [1139]:
app.run(debug=True)
